## Notebook Topic: Getting More Advanced with **pandas**

<ins>Learning Objectives</ins>

1. To analyze a real dataset using more sophisticated **pandas** techniques

**Section I: Loading Data from Python**

Python gives us access to 757 datasets that are nice and clean.  Perfect for learning more **pandas** analysis techniques!  We just need to access the right library.

In [108]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# for sample data, 757 sets
from pydataset import data

# for visualization later!
import seaborn as sns

We can view all the datasets with the following code

In [ ]:
data()

Let's just look at a handful of the datasets and then pick one.

In [ ]:
d = data()
d.head(10) # will give us the first 10 rows of information

In [ ]:
d[:][30:40] # will give us the information at rows 30 to 40

In [ ]:
DF = data('esoph') # we'll select the esoph data
type(DF) # check that it is a pandas dataframe; if not, cast it as a pandas dataframe

Usually, to find more information about functions in Python, we could use the ```help()``` function.  But, to find out more about the datasets in the *pydataset* library, we can't use this method.

Luckily, many of these datasets are from the statistical programming software called **R**.  So, we can find out more information from its [website](https://rdrr.io/r/datasets/esoph.html).

You may need to also search the general internet to find more information about the data.

<span style="color:purple">Use this space in the text box to tell me more about the data.  </span>

**Section II: Exploring a Dataset using pandas**

In this section, we'll explore the *esoph* dataset using the **pandas** library.

**Subsection IIa: Describe and Boxplots**

We may have noticed before that the *esoph* data had three categorical variables (agegp, alcgp, tobgp) and two quantitative variables (ncases, ncontrols).

For quantitative data, we may want to know a summary of its data.  We do this by the ```*.describe()``` function.

In [ ]:
DF.describe()

Notice that ```*.describe()``` only examines the quantitative variables, and it provides an overview of

* count information <-- number of rows in this case
* the mean of the column (or, average number of cases)
* the standard deviation of the column (or, how each data point differs from the average number of cases)
* the minimum value in the column
* the first quartile, which is greater than 25% of the data points
* the second quartile, or median, which is greater than 50% of the data points
* the third quartile, which is greater than 75% of the data points
* the maximum value in the column

In statistics, we call (min, Q1, Q2/Median, Q3, max) the <ins>five number summary</ins>.  From this five number summary, we can say whether there is more data on the lower end (if the median is closer to Q1 than it is to Q3), whether there is more data on the upper end (if the median is closer to Q3 than it is to Q1), or whether the data is roughly symmetric (the first and third quartiles are approximately the same distance from the median).  Let's check!

In [ ]:
diff_lower = DF['ncases'].quantile(0.50) - DF['ncases'].quantile(0.25)
diff_upper = DF['ncases'].quantile(0.75) - DF['ncases'].quantile(0.50)

print('The distance between Q3 and M is ', diff_upper, '\n and the distance between Q1 and M is ', diff_lower)

<span style="color:purple">So, where is the bulk of the data??</span>

We can visualize the same five number summary relatively easy because we are working with a pandas dataframe.  The graph of a five number summary is called a <ins>boxplot</ins>.

In [ ]:
DF.boxplot('ncases')

Notice how the graph demonstrates that *ncases* has more data on the lower end of the numerical spectrum.

**Subsection IIb: Histograms**

Perhaps instead of the five number summary and its boxplot, we want to look at the sample's distribution (or overall shape).  In statistics, we'd use a histogram for that.

In [ ]:
DF.hist('ncases')

The histogram demonstrates, once again, that a majority of the data provided is between 0 and 2.5 cases.  So, this implies to us, the analysts, that most of the data frame contains useless information.

Why?  Because *ncases* is the number of cases of patients that actually had a row's particular combination of (agegp, alcgp, tobgp).  So, it would behoove us to eliminate these rows and make our data smaller for a more meaningful analysis!

**Subsection <span style="color:blue">For You To Create</span>**

Create a new section (above the next subsection but below the *ncases* analysis) in which you analyze the *ncontrols* variable using ```*.describe()``` and ```*.boxplot()``` and ```*.hist()```.

**Subsection IIc: Removing Data**

We will remove all the rows that contain 0 *ncases*.

In [56]:
# identify where in ncases there are 0's
indx = DF[DF['ncases'] == 0].index
# drop those rows
new_DF = DF.drop(indx)

**Subsection IId. Grouping Data**

Now we can analyze (agegp, alcgp, tobgp) more meaningfully.

In [69]:
# sort the rows based on ncases, from largest number to smallest number
top_30_cases = new_DF.sort_values('ncases', ascending = False)
# take the top 30 cases to analyze
top_30_cases = top_30_cases[0:30]
top_30_cases

We can observe that many of these top 30 cases are older.  Let's check in a more efficient way: we will group by age group, and then we will count how many of each age group occurs.

In [ ]:
top_30_cases.groupby(by = ['agegp']).size()

<span style="color:purple">What do you observe??  Insert observations here.</span>

Just like quantitative variables, there are nice ways to graph categorical data.  We like to use barplots.

In [ ]:
# save previous line as variable to be accessed by graphing function
subset = top_30_cases.groupby(by = ['agegp']).size()
# graph the data
subset.plot.bar(x = 'agegp')

<span style="color:purple">What do you observe??  Insert observations here.</span>

The study focuses on patients who have cancer, and they observe their alcohol and tobacco habits.  So, let's see which habit combination is the most frequent!

In [ ]:
# group by alcohol and tobacco habits and count the number of occurrences
sub = top_30_cases.groupby(by = ['alcgp', 'tobgp']).size()
sub

It seems like there aren't combinations that have a significantly higher frequency compared to others.  So, perhaps we can create a graphic that summarizes all observations nicely.

First, notice ```sub``` is not a pandas dataframe.

In [ ]:
type(sub) # not a pandas dataframe

So, to take advantage of DataFrame's abilities, we need to change that.

In [110]:
sub = pd.DataFrame(sub) # cast as pandas dataframe

Now, we'll create a graph called a <ins>heatmap</ins>, which is an $n\times n$ grid, where the rows are notated by one of our variables (we'll use the alcohol consumption) and the columns are notated by the other variable (the tobacco consumption), and the values inside the grid will represent the number of cases of patients who consumes alcohol and tabacco at that level.

In [ ]:
# make a table
top_30_heatmap = sub.pivot_table(index = 'alcgp', columns = 'tobgp')
# visualize the table
sns.heatmap(top_30_heatmap, annot = True)

<span style="color:purple">Describe the graphic here.</span>

**Section III: Synthesizing**

<span style="color:purple">Select a different dataset from the *pydataset* library, and then use the R website to find information on where it came from.  Then insert what you learn below in this text box.</span>

## Conclusion

<span style="color:purple">Summarize what pandas skills you learned in this really, *really* long notebook.</span>

**Note:** You should read chapter 5 for even more information on cool attributes of pandas DataFrames.  Even with this second, longer notebook, we've only scratched the surface.